In [ ]:
# Not needed if tinc-python is installed
import sys
sys.path.append('../external/tinc/tinc-python/tinc-python')
from tinc_client import *
tclient = TincClient()

In [ ]:
tclient.synchronize()

In [ ]:
from parameter import *

In [ ]:
eci1_param=tclient.create_parameter(Parameter,\
                                    "tet_oct_eci","casm",-0.375,0.375,\
                                    [-0.375,-0.125,0.125,0.375],-0.375)

In [ ]:
eci2_param=tclient.create_parameter(Parameter,\
                                    "oct_tet_NN","casm",2.0,6.0,[2.0,6.0],6.0)

In [ ]:
eci3_param=tclient.create_parameter(Parameter,\
                                    "oct_oct_NN","casm",0.0,1.0,\
                                    [0.0,1.0],0.0)

In [ ]:
eci4_param=tclient.create_parameter(Parameter,\
                                    "tet_tet_NN","casm",0.0,0.5,[0.0,0.5])

In [ ]:
tclient.print()

In [ ]:
def create_dir_string_from_eci_param(eci_value):
    datasetname="D:/data_for_visualization/visualization/AMX2_spinel_diffusion_0.0_0.0"+\
    "_"+str(eci1_param.value)+\
    "_"+str(eci2_param.value)+\
    "_"+str(eci3_param.value)+\
    "_"+str(eci4_param.value)+\
    "/kinetic_mc"
    tclient.get_parameter("dataset").value=datasetname
    print("registered change")
    return

In [ ]:
eci1_param.register_callback(create_dir_string_from_eci_param)

eci2_param.register_callback(create_dir_string_from_eci_param)

eci3_param.register_callback(create_dir_string_from_eci_param)

eci4_param.register_callback(create_dir_string_from_eci_param)

In [ ]:
eci1_param.value=-0.125
eci2_param.value=6.0
eci3_param.value=0.0
eci4_param.value=0.0

In [ ]:
print(eci1_param.value)
print(eci2_param.value)
print(eci3_param.value)
print(eci4_param.value)

In [ ]:
shellsites = tclient.get_parameter("ShellSiteTypes")
shellsites.get_current_elements()

In [ ]:
tclient.command_parameter_choice_elements(shellsites)

In [ ]:
tclient.synchronize()

In [ ]:
imageBuffer = tclient.get_disk_buffer('graph')
imageBuffer.allow_cache(5)
imageBuffer._path
imageBuffer.get_filename_for_writing()

In [ ]:
import random
import matplotlib.pyplot as plt
import threading
import shutil
image_names=["oct_doub_va_cleared_path",
            "oct_doub_va_double_cleared_path",
             "oct_doub_va_queued_path",
             "oct_no_va",
             "oct_sing_va",
             "oct_trip_va_forced_path",
             "oct_trip_va_free_range_path",
             "oct_trip_va_semiforced_path",
             "oct_trip_va_trapped_path",
             "tet_no_va",
             "tet_sing_va"]
def update_shellsite_pic(shellSite):
    #print("Shell bitstring " + str(shellSite))
    b = int(shellSite)
    current_image_name=""
    for iname in image_names:
        if b & 1 == 1:
            current_image_name=iname
            break
        b = b >> 1
    with threading.Lock():
        fname = imageBuffer.get_filename_for_writing()
        #print(fname)
        shutil.copyfile("C:/Users/askje/Documents/"+current_image_name+".png",fname)
        imageBuffer.done_writing_file(fname)
        #print(current_image_name)

In [ ]:
shellsites = tclient.get_parameter("ShellSiteTypes")
shellsites.register_callback(update_shellsite_pic)

In [ ]:
param=tclient.parameter_spaces[0]
root_path=param.get_root_path()
trajectories_path=root_path+param.get_current_path()+"\\trajectory.nc"
print(trajectories_path)

In [ ]:
import xarray as xr
import numpy as np

def step_generator():
    trajectories_path=root_path+param.get_current_path()+"\\trajectory.nc"
    print(f'processing: {trajectories_path}')
    return [4,4]
    chunkedby10ds=xr.open_dataset(trajectories_path,chunks={'sample_dim':100})
    #choose an atom to track
    #find the first step that site index changes, record new position
    #track that position from then on
    # repeat if changes. 
    previous_dof=np.array(chunkedby10ds['occupation_dofs'][0][:].values)
    atom_locs=np.where(previous_dof==1)[0].tolist()
    atom_to_track=atom_locs
    print('a')
    steps_changed=[[] for i in range(len(atom_locs))]
    site_labels=[[atom_locs[i]] for i in range(len(atom_locs))]
    previous_dof=np.array(chunkedby10ds['occupation_dofs'][0][:].values)
    for step_count in range(1,3000):
        print(f"step {step_count}")
        new_dof=np.array(chunkedby10ds['occupation_dofs'][step_count][:].values)
        diff_dof=previous_dof-new_dof
        #where diff_dof is 1 this is the site index where atom 
        #disappeared 1->0 
        track_ind=atom_to_track.index(np.where(diff_dof==1)[0][0])
        #time value is 1 more than step count
        steps_changed[track_ind].append(step_count+1)
        #255 means -1 this is the site index where atom 
        #appeared
        atom_to_track[track_ind]=np.where(diff_dof==255)[0][0]
        site_labels[track_ind].append(atom_to_track[track_ind])
        previous_dof=new_dof
    return steps_changed

#for item1,item2 in zip(site_labels,steps_changed):
#    if len(item2)>0:
#        print("trajectory")
#        print(item1)
#        print("time steps")
#        print(item2)

In [ ]:
ps = tclient.get_parameter_space("casm_space_0")
ps.enable_caching()

In [ ]:
ps.get_current_path()

 You can set values you want to try here
 or you can set them from the gui, then run the next cell.
 If you do a parameter sweep, you will need to specify force_values=True
 as the function depends on the server getting the values to produce the 
 current trajectories path

In [ ]:
ps.print()

In [ ]:
ps.get_parameter("time").value = 20

In [ ]:
steps_changed = ps.process(step_generator)

In [ ]:
time_param=tclient.get_parameter("time")
time_param.value
import time
ixes_to_check=[]
for i in range(len(steps_changed)):
    ixes_to_check.append(len(steps_changed[i]))
max_ind=np.argmax(ixes_to_check)
#print(ixes_to_check)
print(steps_changed[max_ind])
#for k in ixes_to_check:
for t in steps_changed[max_ind]:
    time_param.value=t
    time.sleep(0.5)


In [ ]:
ps.get_parameters()